In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import squarify 

In [2]:
def load_scripts():
    df1 = pd.read_csv('./data/simpsons_script_lines.csv', error_bad_lines=False)\
        .sort_values(['episode_id', 'timestamp_in_ms'])\
        .reset_index()
    
    df2 = pd.read_csv('./data/simpsons_episodes.csv', error_bad_lines=False)[['id', 'season']]
    df2.columns = ['episode_id', 'season']

    df3 = pd.merge(df1, df2, how='left', on='episode_id')
    
    return df3


def get_top(df, n_characters):
    # get top characters by line count
    cts = df.raw_character_text.value_counts()
    top = list(cts[:n_characters].index)
    return top


def longest_line(df, character):
    return list(df[np.where(df.raw_character_text==character, True, False)]\
        .sort_values('word_count', ascending=False)['spoken_words'])[0]

In [3]:
df = load_scripts()

b'Skipping line 8084: expected 13 fields, saw 20\nSkipping line 52607: expected 13 fields, saw 21\nSkipping line 59910: expected 13 fields, saw 21\n'
b'Skipping line 71801: expected 13 fields, saw 20\nSkipping line 73539: expected 13 fields, saw 21\nSkipping line 77230: expected 13 fields, saw 21\nSkipping line 78953: expected 13 fields, saw 21\nSkipping line 81138: expected 13 fields, saw 20\nSkipping line 86746: expected 13 fields, saw 22\nSkipping line 101154: expected 13 fields, saw 21\nSkipping line 115438: expected 13 fields, saw 20\nSkipping line 117573: expected 13 fields, saw 22\nSkipping line 130610: expected 13 fields, saw 22\n'
b'Skipping line 152970: expected 13 fields, saw 22\nSkipping line 153017: expected 13 fields, saw 20\nSkipping line 153018: expected 13 fields, saw 30\nSkipping line 154080: expected 13 fields, saw 20\nSkipping line 154082: expected 13 fields, saw 20\nSkipping line 154084: expected 13 fields, saw 20\nSkipping line 154086: expected 13 fields, saw 20\n

In [4]:
len(list(df.raw_text[df.raw_text.str.contains('Washington', na=False)].values)) - len(list(df.raw_text[df.raw_text.str.contains('George Washington', na=False)].values)) -len(list(df.raw_text[df.raw_text.str.contains('Washington D', na=False)].values))

52

In [5]:
states = '''Alabama
Alaska
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Ohio
Oklahoma
Oregon
Pennsylvania
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Virginia
Washington
West Virginia
Wisconsin
Wyoming'''\
.split('\n')


states_abr = '''AL
AK
AZ
AR
CA
CO
CT
DE
FL
GA
HI
ID
IL
IN
IA
KS
KY
LA
ME
MD
MA
MI
MN
MS
MO
MT
NE
NV
NH
NJ
NM
NY
NC
ND
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VT
VA
WA
WV
WI
WY'''\
.split('\n')

In [6]:
states_mentioned = [0]*50
for i,s in enumerate(states):
    if s == 'Washington':
        states_mentioned[i] = len(df.raw_text[df.raw_text.str.contains(s, na=False)].values) -\
            len(df.raw_text[df.raw_text.str.contains('George Washington', na=False)].values) -\
            len(df.raw_text[df.raw_text.str.contains('Washington D', na=False)].values)
        print(states_mentioned[i])
    else:
        states_mentioned[i] = len(df.raw_text[df.raw_text.str.contains(s, na=False)].values)

52


In [7]:
d = {'state': states,
     'mentioned': states_mentioned,
     'code' : states_abr}
df1 = pd.DataFrame.from_dict(d)

In [8]:
print(df1.shape)
df1.head()

(50, 3)


,state,mentioned,code
0,Alabama,6,AL
1,Alaska,28,AK
2,Arizona,7,AZ
3,Arkansas,1,AR
4,California,25,CA


In [9]:
df1.loc[:,'mentioned'] = df1.mentioned.astype('int')

In [10]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
init_notebook_mode(connected=True)

scl = 'Reds'

data = [go.Choropleth(
    colorscale = scl,
    autocolorscale = False,
    locations = df1['code'],
    z = df1['mentioned'].astype(int),
    locationmode = 'USA-states',
    marker = go.choropleth.Marker(
        line = go.choropleth.marker.Line(
            color = 'rgb(255,255,255)',
            width = 2
        )),
    colorbar = go.choropleth.ColorBar(
        title = "Mentions")
)]

layout = go.Layout(
    title = go.layout.Title(
        text = 'How much is each state mentioned on The Simpsons'
    ),
    geo = go.layout.Geo(
        scope = 'usa',
        projection = go.layout.geo.Projection(type = 'albers usa'),
        showlakes = True,
        lakecolor = 'rgb(255, 255, 255)'),
)

fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename = 'd3-cloropleth-map')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~areevesman/0 or inside your plot.ly account where it is named 'd3-cloropleth-map'


/anaconda3/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [11]:
# list(df[np.where(df.raw_text.str.contains('New York'), True, False)].normalized_text)

In [12]:
# set(df.season)

In [13]:
states_by_season = []
seasons = []
for season in set(df.season):
    cond = np.where(df.season==season, True, False)
    states_mentioned = [0]*50
    for i,s in enumerate(states):
        if s == 'Washington':
            states_mentioned[i] = len(df[cond].raw_text[df.raw_text.str.contains(s, na=False)].values) -\
                len(df[cond].raw_text[df[cond].raw_text.str.contains('George Washington', na=False)].values) -\
                len(df[cond].raw_text[df[cond].raw_text.str.contains('Washington D', na=False)].values)
        else:
            states_mentioned[i] = len(df[cond].raw_text[df[cond].raw_text.str.contains(s, na=False)].values)
    j = np.argmax(states_mentioned)
#     print(j, states[j], states_mentioned[j])
    states_by_season.append(states[j])
    seasons.append(season)

d = {'season' : seasons, 'state' : states_by_season}
df2 = pd.DataFrame.from_dict(d)

In [14]:
df2

,season,state
0,1,Iowa
1,2,Michigan
2,3,New York
3,4,Texas
4,5,Florida
5,6,Hawaii
6,7,Washington
7,8,Washington
8,9,New York
9,10,Hawaii


In [15]:
df3 = df2.copy()
df3 = df3.iloc[:-1,:]
df3.columns = ['season', 'start_state']
df3.loc[:,'end_state'] = list(df2.state.values[1:])

In [16]:
df3.head()

,season,start_state,end_state
0,1,Iowa,Michigan
1,2,Michigan,New York
2,3,New York,Texas
3,4,Texas,Florida
4,5,Florida,Hawaii


In [17]:
state_ll = pd.read_csv('./data/statelatlong.csv')[['City','Latitude','Longitude']]
state_ll.columns = ['start_state', 'start_lat', 'start_lon']
state_ll2 = state_ll.copy()
state_ll2.columns = ['end_state', 'end_lat', 'end_lon']
# state_ll.loc[:,'end_state'] = state_ll.start_state.copy()
# state_ll.loc[:,'end_lat'] = state_ll.start_lat.copy()
# state_ll.loc[:,'end_lon'] = state_ll.start_lon.copy()
state_ll2

,end_state,end_lat,end_lon
0,Alabama,32.601011,-86.680736
1,Alaska,61.302501,-158.775020
2,Arizona,34.168219,-111.930907
3,Arkansas,34.751928,-92.131378
4,California,37.271875,-119.270415
5,Colorado,38.997934,-105.550567
6,Connecticut,41.518784,-72.757507
7,Delaware,39.145251,-75.418921
8,District of Columbia,38.899349,-77.014567
9,Florida,27.975728,-83.833017


In [18]:
df4 = pd.merge(df3, state_ll, how='left', on='start_state')
df5 = pd.merge(df4, state_ll2, how='left', on='end_state')
df5.head()

,season,start_state,end_state,start_lat,start_lon,end_lat,end_lon
0,1,Iowa,Michigan,41.938317,-93.389798,44.943560,-86.415805
1,2,Michigan,New York,44.943560,-86.415805,40.705626,-73.979680
2,3,New York,Texas,40.705626,-73.979680,31.169336,-100.076842
3,4,Texas,Florida,31.169336,-100.076842,27.975728,-83.833017
4,5,Florida,Hawaii,27.975728,-83.833017,20.460000,-157.505000


In [28]:
airports = [go.Scattergeo(
    locationmode = 'USA-states',
    lon = df5['start_lon'],
    lat = df5['start_lat'],
    mode = 'markers',
    marker = go.scattergeo.Marker(
        size = 5,
        color = 'rgb(255, 0, 0)',
        line = go.scattergeo.marker.Line(
            width = 3
        )
    ))]

flight_paths = []
for i in range(len(df5)):
    flight_paths.append(
        go.Scattergeo(
            locationmode = 'USA-states',
            lon = [df5['start_lon'][i], df5['end_lon'][i]],
            lat = [df5['start_lat'][i], df5['end_lat'][i]],
            mode = 'lines',
            line = go.scattergeo.Line(
                width = 2,
            ),
#             opacity = float(df5['cnt'][i]) / float(df5['cnt'].max()),
        )
    )

layout = go.Layout(
    title = go.layout.Title(
        text = 'Most popular states in each season of The Simpsons'
    ),
    showlegend = False,
    geo = go.layout.Geo(
        scope = 'north america',
        projection = go.layout.geo.Projection(type = 'orthographic'),
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    )
)

fig = go.Figure(data = flight_paths + airports, layout = layout)
py.iplot(fig, filename = 'd3-flight-paths')